关键ID:

1. hhid: 家庭ID。这是连接其他数据表（如 person 表）的唯一钥匙，必须保留。

2. 家庭构成:

hhsize: 家庭规模。直接反映了家庭的出行需求量。

dwelltype: 住宅类型。公寓和独立屋家庭的出行模式可能有显著差异。(Unit, Flat or Apartment/ Separate House/ Terrace or Townhouse/ other 一共四类)

owndwell: 住房拥有情况。可以间接反映家庭的经济稳定性和生活方式。（Being Rented/ Fully Owned/ Being Purchased/ occupied rent-free 一共四类）

3. 交通资产:

totalvehs: 车辆总数。这是影响出行方式选择的最核心变量之一。

totalbikes: 自行车总数。对于分析短途出行和非机动化出行模式很有价值。

4. 地理与社会经济:

homelga: 家庭所在地方政府区域(LGA)。地理位置是决定出行距离、时间和方式的关键。

hhinc_group: 家庭收入分组。这是另一个核心的社会经济变量，直接影响交通方式的选择和消费能力。（对这一列进行处理，取每年收入的中位数平均值，missing行数的进行这个地区的中位数或者整个列的中位数进行填充）

5. 出行日信息:

dayType: 出行日类型（工作日/周末）。这个变量非常重要，因为人们在工作日和周末的出行行为差异巨大。（分为两种，一种是weekday一种是weekend）



In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('A2 datasets/household_vista_2023_2024.csv')

# 1. 定义需要保留的列
selected_columns = {
    'hhid': 'household_id',
    'hhsize': 'household_size',
    'dwelltype': 'dwelling_type',
    'owndwell': 'dwelling_ownership',
    'totalvehs': 'total_vehicles',
    'totalbikes': 'total_bicycles',
    'homelga': 'lga',
    'hhinc_group': 'income_group',
    'dayType': 'day_type'
}

# 2. 创建只包含这些列的新 DataFrame
household_df = df[selected_columns.keys()].copy()

# 3. 重命名列
household_df.rename(columns=selected_columns, inplace=True)

# 查看结果
print("Selected and renamed columns:")
print(household_df.head())


Selected and renamed columns:
  household_id  household_size            dwelling_type dwelling_ownership  \
0  Y24H5740102               2  Unit, Flat or Apartment       Being Rented   
1  Y24H5740103               1           Separate House        Fully Owned   
2  Y24H5740104               1           Separate House       Being Rented   
3  Y24H5740109               5           Separate House    Being Purchased   
4  Y24H5740110               3           Separate House       Being Rented   

   total_vehicles total_bicycles        lga  \
0               2              0  Casey (C)   
1               1              0  Casey (C)   
2               0              1  Casey (C)   
3               1              5  Casey (C)   
4               1              1  Casey (C)   

                        income_group day_type  
0    $1,250-$1,499 ($65,000-$77,999)  Weekday  
1        $500-$649 ($26,000-$33,799)  Weekday  
2                $1-$149 ($1-$7,799)  Weekday  
3  $2,000-$2,499 ($104,000

In [6]:
# 1. 定义需要转换为数值的列
numeric_cols = ['household_size', 'total_vehicles', 'total_bicycles']

# 2. 进行转换，errors='coerce' 会将无法转换的值变为缺失值(NaN)，（因为这个total_vehicles缺失的表是missing/refused这边就进行处理了，然后）
for col in numeric_cols:
    household_df[col] = pd.to_numeric(household_df[col], errors='coerce')

# 3. 将分类列转换为 'category' 类型，可以节省内存并提高效率
categorical_cols = ['dwelling_type', 'dwelling_ownership', 'lga', 'income_group', 'day_type']
for col in categorical_cols:
    household_df[col] = household_df[col].astype('category')

# 检查转换后的数据类型
print("\nData types after conversion:")
household_df.info()


Data types after conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   household_id        3239 non-null   object  
 1   household_size      3239 non-null   int64   
 2   dwelling_type       3239 non-null   category
 3   dwelling_ownership  3239 non-null   category
 4   total_vehicles      3239 non-null   int64   
 5   total_bicycles      2930 non-null   float64 
 6   lga                 3239 non-null   category
 7   income_group        3150 non-null   category
 8   day_type            3239 non-null   category
dtypes: category(5), float64(1), int64(2), object(1)
memory usage: 119.7+ KB


-这边发现total_bicycles      2930 non-null   float64 有309行是missing
-income_group        3150 non-null   category  有89行是missing
-然后处理的策略是检测total_bicycles和income_group共同缺失的行 假如同时缺失的行小于总数的2%，就drop掉 
-对于total_bicycles处理的策略是 对于缺失的行 就直接填充为0
-对于income_group的收入的策略就是对household_id 用地区进行group，然后用地区的中位数填充缺失的数值，中间需要使用re进行正则提取，处理为中位数，然后再进行缺失值处理

In [7]:
both_na = household_df['total_bicycles'].isna() & household_df['income_group'].isna()

print(household_df.loc[both_na, ])
print(f"BOTH missing count = {both_na.sum()} / {len(household_df)} "
      f"({both_na.mean():.2%})")

     household_id  household_size            dwelling_type dwelling_ownership  \
27    Y24H5740504               2           Separate House    Being Purchased   
679   Y24H5860209               1  Unit, Flat or Apartment        Fully Owned   
1017  Y24H5910433               2           Separate House        Fully Owned   
1211  Y24H5940515               1           Separate House        Fully Owned   
1230  Y24H5950214               1  Unit, Flat or Apartment        Fully Owned   
1288  Y24H5950531               4           Separate House        Fully Owned   
1804  Y24H6040213               2           Separate House        Fully Owned   
1989  Y24H6060313               2  Unit, Flat or Apartment        Fully Owned   
2437  Y24H6126114               1           Separate House        Fully Owned   
2572  Y24H6140337               1     Terrace or Townhouse        Fully Owned   
2674  Y24H6150502               1           Separate House        Fully Owned   
2973  Y24H6200104           

发现toal_bicycles和income_group共同是缺失的行为12行
占比为0.37% 所以可以删去


In [8]:
# 1) 删除同时缺失的行
household_df = household_df.loc[~both_na].copy()

# 2) total_bicycles 缺失填 0，并转为 int(因为nan的field原先数值是missing/refused)
household_df['total_bicycles'] = (
    pd.to_numeric(household_df['total_bicycles'], errors='coerce')
      .fillna(0)
      .astype(int)
)
print(household_df)

     household_id  household_size            dwelling_type dwelling_ownership  \
0     Y24H5740102               2  Unit, Flat or Apartment       Being Rented   
1     Y24H5740103               1           Separate House        Fully Owned   
2     Y24H5740104               1           Separate House       Being Rented   
3     Y24H5740109               5           Separate House    Being Purchased   
4     Y24H5740110               3           Separate House       Being Rented   
...           ...             ...                      ...                ...   
3234  Y24H6250431               3           Separate House       Being Rented   
3235  Y24H6250432               1           Separate House        Fully Owned   
3236  Y24H6250438               4           Separate House    Being Purchased   
3237  Y24H6250440               1           Separate House        Fully Owned   
3238  Y24H6250441               1           Separate House        Fully Owned   

      total_vehicles  total

In [9]:
import pandas as pd
import numpy as np
import re

# 假设 household_df['income_group'] 已存在
s = household_df['income_group'].astype('string').fillna('')

# 1) 统一分隔符/符号
s_norm = (
    s.str.replace("to", "-", regex=False)
     .str.replace("–", "-", regex=False)
     .str.replace("—", "-", regex=False)
     .str.replace("$", "", regex=False)
     .str.replace(",", "", regex=False)
     .str.strip()
)

# 2) 优先解析“括号内”的数字
    #    - 若括号内找到 >=2 个数 => 取前两个数的中位数
    #    - 若括号内只找到 1 个数（如 "416000 or more"）=> 取这个数（low）
def parse_parentheses(val: str):
        m = re.search(r'\(([^)]*)\)', val)
        if not m:
            return np.nan
        inside = m.group(1)
        nums = [float(x.replace(',', '')) for x in re.findall(r'\d[\d,]*\.?\d*', inside)]
        if len(nums) >= 2:
            return (nums[0] + nums[1]) / 2.0
        if len(nums) == 1:
            return nums[0]  # 只有一个下界 => 直接用 low
        return np.nan


income_mid = s_norm.apply(parse_parentheses)

In [10]:
# ==== 将结果并回 DataFrame ====
household_df = household_df.copy()
household_df['income_mid'] = income_mid.astype('float64')

# ==== 1) 覆盖率与缺失 ====
total = len(household_df)
parsed = household_df['income_mid'].notna().sum()
print(f"Parsed rows: {parsed} / {total} ({parsed/total:.2%})")
print(f"NaN rows   : {total - parsed}")

# 看看仍未解析的样本（前 20 条）
nan_rows = household_df.loc[household_df['income_mid'].isna(), ['income_group']]
print("\nExamples of unparsed income_group (up to 20 rows):")
print(nan_rows.head(20))

# ==== 2) 基本分布统计 ====
print("\nDescriptive stats of income_mid:")
print(household_df['income_mid'].describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.95]).round(2))

# 检查是否有非正数（通常不应出现）
num_nonpos = (household_df['income_mid']<=0).sum()
print(f"\nNon-positive incomes: {num_nonpos}")

# ==== 3) 简单分箱查看分布（可选）====
bins = [0, 20000, 40000, 60000, 80000, 100000, 150000, 200000, 300000, 1e9]
labels = ["0-20k","20-40k","40-60k","60-80k","80-100k","100-150k","150-200k","200-300k","300k+"]
binned = pd.cut(household_df['income_mid'], bins=bins, labels=labels, include_lowest=True)
print("\nIncome bins (counts):")
print(binned.value_counts(dropna=True))

# ==== 4) 如需导出结果 ====
household_df.to_csv("household_with_income_mid.csv", index=False)


Parsed rows: 3150 / 3227 (97.61%)
NaN rows   : 77

Examples of unparsed income_group (up to 20 rows):
    income_group
117          NaN
119          NaN
247          NaN
267          NaN
270          NaN
356          NaN
361          NaN
371          NaN
396          NaN
415          NaN
428          NaN
460          NaN
471          NaN
562          NaN
565          NaN
720          NaN
786          NaN
798          NaN
863          NaN
866          NaN

Descriptive stats of income_mid:
count      3150.00
mean     132074.97
std       92785.72
min        3900.00
10%       29899.50
25%       58499.50
50%      116999.50
75%      194999.50
90%      285999.50
95%      363999.50
max      416000.00
Name: income_mid, dtype: float64

Non-positive incomes: 0

Income bins (counts):
income_mid
100-150k    583
150-200k    492
200-300k    411
20-40k      396
40-60k      371
80-100k     359
60-80k      221
300k+       164
0-20k       153
Name: count, dtype: int64


In [13]:
# 1. 使用 groupby().transform() 计算每个家庭所在LGA的收入中位数
lga_median_series = household_df.groupby('lga',observed=True)['income_mid'].transform('median')

# 2. 创建新列，并用计算出的LGA中位数填充缺失值
household_df['income_mid_imputed'] = household_df['income_mid'].fillna(lga_median_series)

分层填充策略：
-对新创建的household表中lga这一列分层，然后用每一个地区的中位数去填充household收入缺失值
-pros：Reduces Analytical Bias
- Leverages Relationships within the Data 相比于单纯的使用全局中位数填充

In [14]:
# 1. 定义最终需要的列
final_columns_to_select = [
    'household_id',
    'household_size',
    'dwelling_type',
    'dwelling_ownership',
    'total_vehicles',
    'total_bicycles',
    'lga',
    'day_type',
    'income_mid_imputed'  # 先选择处理好的收入列
]

# 2. 从您已经处理好的 household_df 中提取这些列
# 使用 .copy() 是一个好习惯，可以避免后续操作出现警告
householdIncome = household_df[final_columns_to_select].copy()

# 3. 将 'income_mid_imputed' 列重命名为 'income'
householdIncome.rename(columns={'income_mid_imputed': 'income'}, inplace=True)

householdIncome.to_csv("householdIncome.csv", index=False)